In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
if not (os.path.exists('./data/userid-profile.tsv') and os.path.exists('./data/userid-timestamp-artid-artname-traid-traname.tsv')):
  raise ValueError('Los archivos userid-profile y userid-timestamp-artid-artname-traid-traname no fueron encontrados en el path')
else:
  print("Los archivos han sido cargados")

In [ ]:
users=pd.read_csv('./data/userid-profile.tsv', sep = '\t', header=0 )

In [ ]:
activity=pd.read_csv('./data/userid-timestamp-artid-artname-traid-traname.tsv', sep = '\t', names = [ 'userid', 'timestamp', 'musicbrainz-artist-id', 'artist-name', 'musicbrainz-track-id', 'track-name' ])

### Primero removemos los 50 usuarios con menos interacciones para reducir el tamaño del dataframe

In [ ]:
activity_lastFifty = activity.groupby('userid')['artist-name'].agg({'count'}).sort_values(by='count', ascending=False).tail(50)

In [ ]:
activity_lastFifty

In [ ]:
activity_lastFifty_union = activity.merge(activity_lastFifty, how='inner', left_on='userid', right_index=True)
activity_lastFifty_union = activity_lastFifty_union[['userid', 'timestamp', 'musicbrainz-artist-id', 'artist-name', 'musicbrainz-track-id', 'track-name']]

In [ ]:
dataframes = [activity,activity_lastFifty_union]
reduced_activity = pd.concat(dataframes).drop_duplicates(keep=False)

In [ ]:
activity.count()

In [ ]:
reduced_activity.count()

### Luego contamos las veces en las que un usuario escuchó un artista

In [ ]:
user_artists_count = reduced_activity.groupby(['userid', 'artist-name']).count()

In [ ]:
sorted_user_artists_count = user_artists_count['track-name'].reset_index(name='count').sort_values(['userid', 'count'], ascending=False)

In [ ]:
test = sorted_user_artists_count[sorted_user_artists_count['userid'] == 'user_001000']

In [ ]:
countmax = test['count'].max()

In [ ]:
test['rating'] = (test['count']/countmax) * 10

### Por último sacamos un rating basado en la cantidad de veces que un artista fué escuchado de manera proporcional